<a href="https://colab.research.google.com/github/VedantDere0104/BERT_Sentiment/blob/main/BERT_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
####

In [2]:
! pip install transformers

In [3]:
import torch

import random
import numpy as np
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id
len(tokenizer.vocab)

30522

In [6]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
max_input_length

512

In [7]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [8]:

from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [9]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [10]:

print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [11]:
print(vars(train_data.examples[6]))

{'text': [2023, 3185, 2001, 7917, 1999, 2563, 1029, 2339, 1029, 3419, 7842, 6371, 2072, 1010, 2577, 18290, 1010, 26800, 23157, 2080, 1010, 12776, 3695, 11865, 15472, 2072, 1998, 2500, 2018, 2589, 2521, 4788, 2077, 1998, 2031, 2506, 2000, 2061, 2144, 1012, 1012, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2023, 3185, 2038, 2035, 1996, 3937, 3787, 1997, 1037, 11519, 17549, 2030, 2220, 3770, 1005, 1055, 5469, 2143, 1012, 2204, 2559, 3057, 1006, 2040, 2064, 1005, 1056, 2552, 2000, 3828, 2037, 3268, 1010, 2011, 1996, 2126, 1007, 1010, 1037, 6659, 7407, 4040, 2007, 1037, 22047, 19909, 2091, 27757, 1010, 1037, 8040, 26688, 1010, 1037, 4689, 2567, 13071, 2105, 1996, 2155, 3776, 1010, 1998, 2941, 1037, 3492, 4365, 2204, 9792, 2012, 1996, 2203, 1012, 2021, 7917, 1029, 5667, 1012, 2043, 1996, 2394, 3323, 7917, 2023, 3185, 1010, 1996, 16773, 2763, 4191, 2037, 7268, 4632, 2229, 2125, 2012, 2129, 11043, 1998, 10975, 21041, 4095, 1996, 28101, 2015, 2428, 2020, 1012, 1026, 7987, 1013, 1028, 

In [12]:

tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['this', 'movie', 'was', 'banned', 'in', 'england', '?', 'why', '?', 'tom', 'sa', '##vin', '##i', ',', 'george', 'romero', ',', 'dario', 'argent', '##o', ',', 'luc', '##io', 'fu', '##lc', '##i', 'and', 'others', 'had', 'done', 'far', 'worse', 'before', 'and', 'have', 'continued', 'to', 'so', 'since', '.', '.', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'this', 'movie', 'has', 'all', 'the', 'basic', 'elements', 'of', 'a', 'decent', '70s', 'or', 'early', '80', "'", 's', 'horror', 'film', '.', 'good', 'looking', 'girls', '(', 'who', 'can', "'", 't', 'act', 'to', 'save', 'their', 'lives', ',', 'by', 'the', 'way', ')', ',', 'a', 'terrible', 'lightning', 'storm', 'with', 'a', 'torre', '##ntial', 'down', '##pour', ',', 'a', 'sc', '##ythe', ',', 'a', 'crazy', 'brother', 'wandering', 'around', 'the', 'family', 'estate', ',', 'and', 'actually', 'a', 'pretty', 'damn', 'good', 'twist', 'at', 'the', 'end', '.', 'but', 'banned', '?', 'seriously', '.', 'when', 'the', 'english', 'parliament', 'ban

In [13]:
LABEL.build_vocab(train_data)


In [14]:

print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fae08d20d90>, {'neg': 0, 'pos': 1})


In [15]:

BATCH_SIZE = 128

device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [16]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [17]:

import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [18]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [20]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [21]:

for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


In [22]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [23]:

criterion = nn.BCEWithLogitsLoss()

In [24]:
model = model.to(device)
criterion = criterion.to(device)

In [25]:

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [26]:

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [32]:
from tqdm.auto import tqdm
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in tqdm(iterator):

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [30]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/Relatas_Assignment/tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 15m 45s
	Train Loss: 0.319 | Train Acc: 86.85%
	 Val. Loss: 0.265 |  Val. Acc: 89.30%
Epoch: 02 | Epoch Time: 15m 47s
	Train Loss: 0.246 | Train Acc: 90.18%
	 Val. Loss: 0.234 |  Val. Acc: 90.76%
Epoch: 03 | Epoch Time: 15m 46s
	Train Loss: 0.214 | Train Acc: 91.70%
	 Val. Loss: 0.227 |  Val. Acc: 90.91%
Epoch: 04 | Epoch Time: 15m 46s
	Train Loss: 0.189 | Train Acc: 92.73%
	 Val. Loss: 0.227 |  Val. Acc: 91.18%
Epoch: 05 | Epoch Time: 15m 44s
	Train Loss: 0.168 | Train Acc: 93.51%
	 Val. Loss: 0.259 |  Val. Acc: 90.61%


In [33]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Relatas_Assignment/tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Test Loss: 0.204 | Test Acc: 91.73%


In [34]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [35]:
predict_sentiment(model, tokenizer, "This film is terrible")

0.028803542256355286

In [36]:
predict_sentiment(model, tokenizer, "This film is great")

0.9502979516983032

In [42]:
predict_sentiment(model, tokenizer, "We are greatly appreciative of all of ZS's hard work and cooperation with us on this project.")

0.580721914768219